# Processamento de Linguagem Natural (PLN) - Básico

SERPRO - SUPSD - DIVISÃO DE DESENVOLVIMENTO E SUSTENTAÇÃO DE PRODUTOS COGNITIVOS


## Parte 3: Exemplo - Classificação de Sentenças com LSTM

Redes Neurais Recorrentes (RNNs - Recurrent Neural Networks) são uma poderosa família de redes neurais que são amplamente utilizadas para tarefas de modelagem de sequência (por exemplo, previsão de preço de ações, modelagem de linguagem, séries temporais, etc...). A capacidade das RNNs de explorar dependências temporais de entidades em uma sequência as torna poderosas. Neste exercício, utilizaremos uma LSTM, que é uma RNN com células de memória para classificação de sentenças.

Na classificação da sentenças, uma dada sentença deve ser classificada em uma classe (categoria). Usaremos um banco de dados de perguntas, onde cada pergunta é rotulada pelo assunto da pergunta. Por exemplo, a pergunta "Quem foi Abraham Lincoln?" será uma pergunta e seu rótulo será Pessoa. Para isso, usaremos um conjunto de dados de classificação de frases disponível em http://cogcomp.org/Data/QA/QC/; aqui você encontrará sentenças de treinamento e seus respectivos rótulos e 500 sentenças de teste.

Categorias: http://cogcomp.org/Data/QA/QC/definition.html

Usaremos a seguinte estratégia:  
 - Extrair os dados
 - Pré-processamento dos dados
 - Construir a Rede Neural LSTM
 - Treinamento do modelo
 - Avaliar o modelo
 - Realizando Predições com dados de Teste

In [1]:
#Imports
import os
import numpy as np 
import pandas as pd
import re
import operator

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences

from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [2]:
tf.__version__

'1.13.1'

## Verificando o Dataset

Usaremos o __[dataset](http://cogcomp.cs.illinois.edu/Data/QA/QC/)__ que é composto de perguntas como entradas e seu respectivo tipo como saída. Por exemplo: quem era Abraham Lincon? E a saída ou rótulo seria Humano.


In [3]:
# Diretório
dir_name = 'data/question-classif-data'

# Dataset de treino
#train_filename = 'train_1000.label'
train_filename = 'train_5500.label'

# Dataset de teste
test_filename = 'TREC_10.label'

In [4]:
# Verifica se os arquivos existem
filenames = [train_filename,test_filename]
num_files = len(filenames)
for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name,filenames[i]))
    assert file_exists
print('Arquivos encontrados e verificados!')

Arquivos encontrados e verificados!


## Carregando e visualizando os dados

Abaixo nós carregamos o texto e visualizar alguns dados

In [5]:
# Função para leitura dos dados
def read_data_into_pandas(filename):
    col_names =  ['Category', 'Subcategory', 'Question']
    data  = pd.DataFrame(columns = col_names)
    with open(filename,'r',encoding='latin-1') as f: 
        count=1;
        for row in f:
            #Quebra a linha em duas no primeiro espaço (category:subcategory and question)
            row_str = row.split(" ",1)
            #Separa a categoria da subcategoria
            row_category = row_str[0].split(":")            
            data.loc[len(data)] = [row_category[0],row_category[1],row_str[1].lower()]
            count+=1
    return data
# Carrega os dados em DataFrames específicos para treinamento e teste
data_train = read_data_into_pandas(os.path.join(dir_name,train_filename))
data_test = read_data_into_pandas(os.path.join(dir_name,test_filename))

# Concatena os dados de treinamento e teste. Será utilizado para construir um vocabulário único
data = pd.concat([data_train,data_test])
print(data.head())

  Category Subcategory                                           Question
0     DESC      manner  how did serfdom develop in and then leave russ...
1     ENTY      cremat  what films featured the character popeye doyle...
2     DESC      manner  how can i find a list of celebrities ' real na...
3     ENTY      animal  what fowl grabs the spotlight after the chines...
4     ABBR         exp                  what is the full form of .com ?\n


In [6]:
# Visualizando algumas informações sobre os dados
# Exemplo Treinamento
print('Treinamento')
print(' - Número questões na categoria HUMAN : {}'.format(data_train[data_train['Category'] == 'HUM'].size))
print(' - Exemplos de questões na categoria HUMAN :')
print(data_train[data_train['Category'] == 'HUM']['Question'].head())
print('')
# Exemplo Teste
print('Teste')
print(' - Número questões na categoria HUMAN : {}'.format(data_test[ data_test['Category'] == 'HUM'].size))
print(' - Exemplos de questões na categoria HUMAN :')
print(data_test[data_test['Category'] == 'HUM']['Question'].head())

Treinamento
 - Número questões na categoria HUMAN : 3669
 - Exemplos de questões na categoria HUMAN :
5     what contemptible scoundrel stole the cork fro...
6     what team did baseball 's st. louis browns bec...
7                     what is the oldest profession ?\n
9     name the scar-faced bounty hunter of the old w...
12                 who was the pride of the yankees ?\n
Name: Question, dtype: object

Teste
 - Número questões na categoria HUMAN : 195
 - Exemplos de questões na categoria HUMAN :
2                                   who was galileo ?\n
6     george bush purchased a small interest in whic...
11                 what person 's head is on a dime ?\n
13    who was the first man to fly across the pacifi...
17      who developed the vaccination against polio ?\n
Name: Question, dtype: object


## Pré-processamento e Vetorização dos Dados para o Treinamento

Utilizaremos a classe Tokenizer do __[Keras](https://keras.io/)__ para montar nosso vocabulário. Ela permite vetorizar o texto do corpus, transformando o texto na sequência de inteiros (cada inteiro representa o índice do token em um dicionário) 
https://keras.io/preprocessing/text/

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Question'].values)
print("Tamanho do Vocabulário: ", len(tokenizer.word_index))
print("10 primeiras palavras do vocabulário: ")
iterator = iter(tokenizer.word_index.items())
for i in range(10):
    print(next(iterator))

Tamanho do Vocabulário:  8761
10 primeiras palavras do vocabulário: 
('the', 1)
('what', 2)
('is', 3)
('of', 4)
('in', 5)
('a', 6)
('how', 7)
("'s", 8)
('was', 9)
('who', 10)


In [8]:
# Vetorização dos dados de treinamento
train_X = tokenizer.texts_to_sequences(data_train['Question'].values)
print("Exemplo de frase vetorizada:", train_X[0])
print("Exemplo de frase vetorizada:", train_X[1])
print("Exemplo de frase vetorizada:", train_X[3])

# Padding dos dados de treinamento: preencher com zeros para que todas as frases fiquem do mesmo tamanho
# Necessário para alimentar a rede neural  
train_X = pad_sequences(train_X,value=0.)

print("Exemplo de frase vetorizada preenchida com zeros: ", train_X[0])
print("Shape do vetor de treinamento:",train_X.shape)
print("Tamanho da maior frase:",train_X.shape[1])

# Vetorização dos labels para os dados de treinamento
train_labels = pd.get_dummies(data_train['Category'])
print("Categorias: ",train_labels.columns)
train_Y = pd.get_dummies(data_train['Category']).values
print("Exemplo vetor de label:",train_Y[0])


Exemplo de frase vetorizada: [7, 15, 3656, 2318, 5, 14, 440, 852, 1033]
Exemplo de frase vetorizada: [2, 853, 639, 1, 134, 1319, 3657]
Exemplo de frase vetorizada: [2, 3658, 3659, 1, 3660, 137, 1, 499, 46, 4, 1, 1683]
Exemplo de frase vetorizada preenchida com zeros:  [   7   15 3656 2318    5   14  440  852 1033    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
Shape do vetor de treinamento: (5452, 33)
Tamanho da maior frase: 33
Categorias:  Index(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype='object')
Exemplo vetor de label: [0 1 0 0 0 0]


In [9]:
# Vetorização dos dados de teste
test_X = tokenizer.texts_to_sequences(data_test['Question'].values)

# Temos que usar o mesmo tamanho de frases pros dados de testes, por isso usamos o tamanho dos dados de treinamento
test_X = pad_sequences(test_X,maxlen=train_X.shape[1],value=0.)
print("Shape do vetor de teste",test_X.shape)
test_labels = pd.get_dummies(data_test['Category']).columns
test_Y = pd.get_dummies(data_test['Category']).values

Shape do vetor de teste (500, 33)


## Construção da Rede Neural (LSTM)
http://tflearn.org/layers/recurrent/#lstm

Vamos criar uma rede LSTM com a seguinte arquitetura:
- O input será uma matriz com dimensão X (onde x é maior número de palavras de um documento)
- Camada de entrada terá o número de neurônio igual ao número de features (words embeddings) 
- Uma camada LSTM com 128 unidades nas camadas ocultas nas células LSTM. Geralmente quanto maior melhor é desempenho. Os valores comuns são 128, 256, 512, etc.
- Camada Dropout para evitar o overfitting
- E uma camada totalmente conectada com 6 neurônios, referente as quantidade de categorias, função de ativação Softmax com as probabilidades para cada classe
- Por fim uma camada de regressão com a função de perda e otimização

In [10]:
# Reset do grafo tenforflow
tf.reset_default_graph()

# Construção da Rede
network = tflearn.input_data([None, train_X.shape[1]])
network = tflearn.embedding(network, input_dim=len(tokenizer.word_index)+1, output_dim=128)
network = tflearn.lstm(network, 128, dropout=0.5)
network = tflearn.fully_connected(network, 6, activation='softmax')
network = tflearn.regression(network, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')
model = tflearn.DNN(network, tensorboard_verbose=0)

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


## Treinamento e Avaliação do Modelo

Para treinar o modelo vamos utilizar 90% dos dados para treinamento e 10% para validação. 

Lembrado que os hiperparâmetros do treinamento podem ser ajustados de acordo com a necessidade e com o intuito de melhorar a acurácia.

In [11]:
model.fit(train_X, train_Y, n_epoch = 20,validation_set=0.1,batch_size=32,shuffle=True, show_metric=True)

Training Step: 3079  | total loss: 0.05078 | time: 6.962s
| Adam | epoch: 020 | loss: 0.05078 - acc: 0.9893 -- iter: 4896/4906
Training Step: 3080  | total loss: 0.04804 | time: 8.005s
| Adam | epoch: 020 | loss: 0.04804 - acc: 0.9903 | val_loss: 0.89458 - val_acc: 0.8095 -- iter: 4906/4906
--


Avaliando o modelo com os dados de teste.

In [12]:
model.evaluate(test_X,test_Y)

[0.8420000014305115]

## Realizando Predições com dados de Teste

Vamos visualizar algumas predições dos dados de teste

In [13]:
# Predição dos dados de testes
predictions = model.predict(test_X)

# Criação do dicionário reverso (key -> Índice ; value -> Token)
reverse_dictionary = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))

# Varre os dados de teste e imprime a predição
for i in range(len(test_X[:5])):
    frase='';
    for j in test_X[i]:
        if j != 0:
            frase+=reverse_dictionary[j]+' '
        else:
            break
    print('Frase ->',frase)
    print('    Label category->',data_test['Category'][i])
    print('    Label predict->', test_labels[np.argmax(predictions[i])])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

Frase -> how far is it from denver to aspen 
    Label category-> NUM
    Label predict-> NUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> what county is modesto california in 
    Label category-> LOC
    Label predict-> LOC
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> who was galileo 
    Label category-> HUM
    Label predict-> HUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> what is an atom 
    Label category-> DESC
    Label predict-> DESC
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> when did hawaii become a state 
    Label category-> NUM
    Label predict-> NUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Realizando predições com um dado qualquer

In [14]:
test = tokenizer.texts_to_sequences(["where is Brazil?"])
test = pad_sequences(test,maxlen=train_X.shape[1],value=0.)

predictions = model.predict(test)
print(predictions)
print('Label predict->', test_labels[np.argmax(predictions[0])])   

[[2.8468869e-03 6.1455142e-04 4.6957177e-03 3.3550777e-03 9.8799729e-01
  4.9038429e-04]]
Label predict-> LOC


## Exercícios

---

### Exercício 1

Modifique este jupyter para que o modelo obtenha um melhor desempenho.

Primeiro crie uma função de pré-processamento dos dados para realizar as seguintes tarefas:

- Remoção de Pontuação
- Tokenização
- Remoção de Stop words
- Aplicação de Stemmer

Dica: Essa função foi criada e utilizada no jupyter anterior.

Reprocesse e veja os resultados.

---

---

### Exercício 2

Faça modificando para tentar melhorar a acurácia do modelo:

- Uso de Lemmatization ao invés de Stemming
- Modificações dos hiperparâmetros da rede neural, tais como 'n_epoch' e 'batch_size'
- Modifique a arquitetura da rede neural, tal como a quantidade de neurônios

Qual configuração obteve melhores resultados?

---

## Fim